In [ ]:
from implementations.tools.vector_search_tool import VectorSearchTool
from llm_client.llm_client import LLMProcessor
from llm_client.document_vector_store import (
    DocumentStore,
    VectorStore
)

from config.settings import settings
import json
from typing import Dict,Optional,Any
from implementations.tools.list_selected_documents_tool import \
    ListSelectedDocumentsTool
from implementations.tools.read_documents_tool import ReadDocumentsTool
from implementations.tools.vector_search_tool import VectorSearchTool
from implementations.tools.document_relevance_tool import DocumentRelevanceTool
from llm_client.agent import MultiTurnAgent
from llm_client.prompt_builder import  PromptBuilder
from llm_client.document_vector_store import (
    EmbeddingProcessor,
    DocumentStore,
    VectorStore
)
from config.settings import Settings
from content_retrieval import llm,doc_store ,vector_store
from interface.project import Project
settings = Settings()

project  = Project("Hallo",[],123)

def streamlit_tool_callback(tool_call: Dict[str, Any], project: Project):
    """Callback die de UI van een specifiek project bijwerkt."""
    function_name = tool_call.get('function', {}).get('name')
    try:
        args = json.loads(tool_call.get('function', {}).get('arguments', '{}'))
    except json.JSONDecodeError:
        args = {}

    if function_name == "update_document_shortlist":
        for score_info in args.get("scores", []):
            doc_id = score_info.get('document_id')
            project.upsert_document(doc_id,score_info.get('score'))

    elif function_name == "update_scratchpad":
        project.scratchpad = args.get("tasks", [])
        
def list_documents_callback(tool_result: Dict[str, Any], project):
    """
    Callback function for the list_selected_documents tool.
    Returns the list of all selected documents (both agent-found and user-found).
    """
    all_documents = {
        "agent_found": list(project.agent_found_documents.keys()),
        "user_found": list(project.self_found_documents.keys())
    }

    return json.dumps(all_documents)       

def streamlit_tool_result_callback(tool_result: Dict[str, Any], project: Project) -> Optional[str]:
    """Callback die de resultaten van een tool verwerkt voor een specifiek project."""
    function_name = tool_result.get("function_name")

    if function_name == "vector_search":
        try:
            documents = json.loads(tool_result.get("output", "[]"))
            if not isinstance(documents, list):
                return None
            for doc_info in documents:
                doc_id = doc_info.get('id')
                project.upsert_document(doc_id=doc_id)
        except (json.JSONDecodeError, AttributeError):
            pass
    
    return None

on_result_with_project = lambda tool_result: streamlit_tool_result_callback(tool_result, project)
embed_client_config = settings.clients.get(settings.embedding_client_map.get(settings.embedding_model))

def load_heavy_components():
    """Laadt de componenten die niet afhankelijk zijn van de project-staat."""
    print("Initializing heavy components (LLM, Embedder, Stores)...")

    # --- LLM Processor Setup ---
    llm_client_name = settings.llm_client_map.get(settings.llm_model)
    if not llm_client_name or llm_client_name not in settings.clients:
        raise ValueError(f"Client '{llm_client_name}' for model '{settings.llm_model}' not found or configured in settings.")

    # Get the client config dict and make a copy to avoid side effects
    llm_config_dict = settings.clients[llm_client_name].copy()
    llm_config_dict['type'] = 'azure' if 'azure' in llm_client_name else llm_client_name

    llm = LLMProcessor(
        model=settings.llm_model,
        client_config=llm_config_dict
    )

    # --- Embedding Processor Setup ---
    embedding_client_name = settings.embedding_client_map.get(settings.embedding_model)
    if not embedding_client_name or embedding_client_name not in settings.clients:
        raise ValueError(f"Client '{embedding_client_name}' for model '{settings.embedding_model}' not found or configured in settings.")
        
    embedding_config_dict = settings.clients[embedding_client_name].copy()
    embedding_config_dict['type'] = 'azure' if 'azure' in embedding_client_name else embedding_client_name

    embedder = EmbeddingProcessor(
        embedding_model=settings.embedding_model,
        client_config=embedding_config_dict
    )


    doc_store = DocumentStore(
        settings.raw_doc_store_name,
        settings.docstore_folder,
        settings.indexed_metadata_keys
    )
    vector_store = VectorStore(embedder=embedder,
                               doc_store=doc_store,
                               data_root=settings.docstore_folder)

    return llm, doc_store, vector_store

def initialize_agent_for_project(project: Project, llm: LLMProcessor, vector_store: VectorStore, doc_store : DocumentStore) -> MultiTurnAgent:
    """Initialiseert en configureert de agent voor een specifiek project."""
    
    on_call_with_project = lambda tool_call: streamlit_tool_callback(tool_call, project)
    on_result_with_project = lambda tool_result: streamlit_tool_result_callback(tool_result, project)
    on_list_documents = lambda tool_result: list_documents_callback(tool_result, project)

    vs_tool = VectorSearchTool(
        vector_store=vector_store,
        on_result=on_result_with_project
    )
    shortlist_tool = DocumentRelevanceTool(
        on_call=on_call_with_project
    )
    list_tool = ListSelectedDocumentsTool(
        on_result=on_list_documents
    )
    read_tool = ReadDocumentsTool(
        doc_store=doc_store
    )

    agent = MultiTurnAgent(
        llm_processor=llm,
        prompt_processor=PromptBuilder('prompt_templates', 'search'),
        tools=[vs_tool, shortlist_tool, list_tool, read_tool],
        messages=project.messages
    )
    return agent
llm , doc_store ,vector_store = load_heavy_components()
agent = initialize_agent_for_project(project,llm,vector_store,doc_store)
results = agent.chat(query="Use vector_search tool to search for 'Hoe doe ik inkomstenbelasting'",max_tool_turns=999)
# vs = VectorSearchTool(vector_store=vector_store,on_result=on_result_with_project)
# args = {
#   "queries": [
#     "aangifte omzetbelasting indienen",
#     "termijn aangifte btw indienen",
#     "gevolgen te late aangifte btw boete en aansprakelijkheid",
#     "reactietermijn Belastingdienst verwerking aangifte omzetbelasting",
#     "hoe aangifte indienen Belastingdienst DigiD of digipoort",
#     "uitstel of verlenging aangifte omzetbelasting aanvragen",
#     "aangifte corrigeren of intrekken omzetbelasting"
#   ],
#   "n_results": 7
# }
# vs.execute(**args)
